In [3]:

import torch
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import pandas as pd
import torch
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import DataLoader
import os

'''
human keypoints: 17*2
court keypoints: 6*2
net keypoints:   4*2
ball keypoints:  1*2
total keypoints: 28*2

'''




class HitDataset(Dataset):
    def __init__(self, dataset_folder,num_consecutive_frames,normalization=True):
        self.dataset=[]
        self.positive=0
        self.negative=0
        # 遍历文件夹及其子文件夹，找到所有的CSV文件路径
        cnt=0
        for root, dirs, files in os.walk(dataset_folder):
            for file in files:
                if file.endswith(".csv"):
                    # 定义处理函数
                    data_path=os.path.join(root, file)
                    print(data_path)
                    
                    df= pd.read_csv(data_path, converters={"ball": eval,"top":eval,"bottom":eval,"court":eval,"net":eval})
                    
                    rows = len(df)
                    remainder = rows % 12
                    if remainder > 0:
                        num_to_pad = 12 - remainder
                    else:
                        num_to_pad = 0

                    if num_to_pad > 0:
                        last_row = df.iloc[-1]
                        padding_data = np.tile(last_row.values, (num_to_pad, 1))
                        padded_df = pd.DataFrame(padding_data, columns=df.columns)
                        df = pd.concat([df, padded_df], axis=0)
                        df = df.reset_index(drop=True)

                    small_dataset =df
                    
                    for i in range(len(small_dataset)):
                        if i%num_consecutive_frames!=0:
                            continue
                        if i>=len(small_dataset)-num_consecutive_frames:
                            break
                        oridata=small_dataset.loc[i:i+num_consecutive_frames-1,:].copy()
                        oridata=oridata.reset_index(drop=True)
                        data=[]
                        
                        target1=None
                        
                        for index,row in oridata.iterrows():
                            pos=np.array(row['pos'])
                            if str(pos)=='top': 
                                if target1 is None:
                                    target1=[0.0,1.0,0.0]
                                    self.positive+=1
                            elif str(pos)=='bottom':
                                if target1 is None:
                                    target1=[0.0,0.0,1.0]
                                    self.positive+=1
                                    
                        
                            top=np.array(row['top']).reshape(-1,2)
                            bottom=np.array(row['bottom']).reshape(-1,2)
                            court=np.array(row['court']).reshape(-1,2)
                            # net=np.array(row['net']).reshape(-1,2)
                            ball=np.array(row['ball']).reshape(-1,2)
                            
                            frame_data = np.concatenate((top, bottom, court, ball), axis=0)
                            
                            if normalization:
                                frame_data[:,0]/=1920
                                frame_data[:,1]/=1080
                            data.append(frame_data.reshape(1,-1))
                        data=np.array(data)
                        if target1 is None:
                            if self.negative>self.positive:
                                continue
                            target1=[1.0,0.0,0.0] 
                            self.negative+=1                        
                        self.dataset.append((data.reshape(-1),target1))
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        # 假设每个样本是一个元组 (input, target)
        sample = self.dataset[index]
        input_data = sample[0]
        target1 = sample[1]
        
        # 转换为Tensor对象
        input_tensor = torch.tensor(input_data)
        target1_tensor = torch.tensor(target1)
        
        return input_tensor, target1_tensor
    

num_consecutive_frames=12
batch_size=30
shuffle=True
normalization=True

TrainDataset=HitDataset("./HitDataset/train",num_consecutive_frames,normalization)
ValidDataset=HitDataset("./HitDataset/valid",num_consecutive_frames,normalization)



./HitDataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-1.csv
./HitDataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-11.csv
./HitDataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-12.csv
./HitDataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-14.csv
./HitDataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-16.csv
./HitDataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-17.csv
./HitDataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-18.csv
./HitDataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-19.csv
./HitDataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-2.csv
./HitDataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022

In [6]:
class HitModel(nn.Module):
    def __init__(self, feature_dim, num_consecutive_frames):
        super(HitModel, self).__init__()
        self.num_consecutive_frames = num_consecutive_frames
        self.feature_dim = feature_dim
        
        self.gru1 = nn.GRU(feature_dim // num_consecutive_frames, 64, bidirectional=True, batch_first=True)
        self.gru2 = nn.GRU(128, 64, bidirectional=True, batch_first=True)
        self.global_maxpool = nn.MaxPool1d(num_consecutive_frames)
        self.dense = nn.Linear(128, 3)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        batch_size = x.shape[0]
        x=x.float()
        x = x.view(batch_size, self.num_consecutive_frames, self.feature_dim // self.num_consecutive_frames)
        x, _ = self.gru1(x)
        x, _ = self.gru2(x)
        x = x.transpose(1, 2)
        x = self.global_maxpool(x).squeeze()
        x= self.dense(x)
        x=self.softmax(x)
        return x

print(TrainDataset.positive,TrainDataset.negative)
print(ValidDataset.positive,ValidDataset.negative)

feature_dim=82*num_consecutive_frames
train_data_loader = DataLoader(TrainDataset, batch_size=batch_size, shuffle=shuffle)

valid_data_loader = DataLoader(ValidDataset, batch_size=batch_size, shuffle=shuffle)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=HitModel(feature_dim,num_consecutive_frames)
criterion = nn.CrossEntropyLoss()
model.to(device)
criterion.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 100



train_loss_list = []
valid_loss_list=[]
for epoch in range(num_epochs):
    train_loss_sum=0
    model.train()
    for batch_data in train_data_loader:
        inputs, labels = batch_data
        inputs = inputs.to(device)  # 将输入数据移动到设备上
        labels = labels.to(device)  # 将输入数据移动到设备上
        
        outputs = model(inputs)
        outputs=outputs.reshape(-1,3)
        
        train_loss = criterion(outputs,labels)
        train_loss_sum+=train_loss.detach() 
        # 反向传播和优化
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        # 执行你的训练或测试操作
    train_loss_list.append(train_loss_sum)

        
    # 打印训练信息
    if (epoch + 1) % 1== 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs,
                                                    train_loss.item()))
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in valid_data_loader:
            inputs = inputs.to(device)  # 将输入数据移动到设备上
            labels=labels.to(device)
            # 前向传播
            outputs = model(inputs)
            outputs=outputs.reshape(-1,3)
        
            
            y_true=torch.argmax(labels,axis=1)
            y_pred=torch.argmax(outputs,axis=1)
            total+=len(y_true)
            correct+=(y_true==y_pred).sum().item()
        if total==0:
            print(f'Accuracy on test set: {0}')
            continue
        
            
        
        print(f'Accuracy on test set: {correct/total}')

17009 17009
1632 1629
Epoch [1/100], Loss: 0.9626
Accuracy on test set: 0.6596136154553818
Epoch [2/100], Loss: 0.8304
Accuracy on test set: 0.7264642747623429
Epoch [3/100], Loss: 0.7969
Accuracy on test set: 0.7034651947255444
Epoch [4/100], Loss: 0.8980
Accuracy on test set: 0.7322907083716651
Epoch [5/100], Loss: 0.8672
Accuracy on test set: 0.697638761116222
Epoch [6/100], Loss: 0.9153
Accuracy on test set: 0.7387304507819688
Epoch [7/100], Loss: 0.7838
Accuracy on test set: 0.7525298988040479
Epoch [8/100], Loss: 0.7647
Accuracy on test set: 0.7589696412143514
Epoch [9/100], Loss: 0.7343
Accuracy on test set: 0.761116222017786
Epoch [10/100], Loss: 0.7040
Accuracy on test set: 0.7525298988040479
Epoch [11/100], Loss: 0.7954
Accuracy on test set: 0.661146887457835
Epoch [12/100], Loss: 0.6555
Accuracy on test set: 0.7660226924256363
Epoch [13/100], Loss: 0.7586
Accuracy on test set: 0.7552897884084636
Epoch [14/100], Loss: 0.8557
Accuracy on test set: 0.7660226924256363
Epoch [15/

In [7]:
# 保存整个模型
torch.save(model, 'hitdetect.pth')
# print(f"best_accuracy={best_accuracy}")
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in valid_data_loader:
        inputs = inputs.to(device)  # 将输入数据移动到设备上
        labels = labels.to(device)  # 将标签移动到设备上
        
        # 前向传播
        outputs = model(inputs)
        
        y_true=torch.argmax(labels,axis=1)
        y_pred=torch.argmax(outputs,axis=1)
        print(y_true)
        print(y_pred)
        total+=len(y_true)
        correct+=(y_true==y_pred).sum().item()
        
    
    print(f'Accuracy on test set: {correct/total}')




tensor([0, 1, 2, 1, 0, 0, 2, 2, 0, 0, 0, 2, 1, 0, 2, 0, 2, 0, 0, 0, 0, 2, 1, 0,
        1, 1, 0, 0, 0, 0], device='cuda:0')
tensor([2, 1, 0, 1, 0, 0, 2, 2, 0, 2, 0, 0, 1, 0, 2, 2, 0, 0, 0, 1, 0, 2, 0, 0,
        1, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 1, 1, 2, 2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 2, 1, 0, 0, 2, 0, 0, 0, 0, 2,
        0, 0, 0, 2, 0, 2], device='cuda:0')
tensor([0, 0, 1, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2,
        2, 0, 0, 0, 0, 2], device='cuda:0')
tensor([0, 0, 1, 1, 0, 0, 2, 0, 1, 2, 1, 0, 0, 2, 2, 2, 0, 2, 2, 2, 1, 0, 0, 2,
        1, 2, 0, 2, 0, 2], device='cuda:0')
tensor([0, 0, 0, 1, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0, 2, 0, 2, 0, 0, 0, 2,
        1, 2, 0, 2, 2, 2], device='cuda:0')
tensor([0, 0, 1, 2, 2, 0, 1, 0, 2, 2, 1, 2, 2, 2, 0, 1, 0, 0, 0, 0, 0, 2, 2, 0,
        1, 1, 0, 1, 1, 2], device='cuda:0')
tensor([0, 0, 0, 2, 2, 0, 0, 0, 1, 2, 1, 0, 1, 2, 0, 1, 0, 0, 0, 0, 0, 2, 2, 0,
        1, 1, 0, 1, 2, 2], device='cuda:0')
tensor([

In [8]:
final_list=[]
true_list=[]
data_path="./HitDataset/test\Viktor_AXELSEN_Lakshya_SEN_YONEX_All_England_Open_2022_Finals/rally_2-3.csv"
df= pd.read_csv(data_path, converters={"ball": eval,"top":eval,"bottom":eval,"court":eval,"net":eval})

rows = len(df)
remainder = rows % 12
if remainder > 0:
    num_to_pad = 12 - remainder
else:
    num_to_pad = 0

if num_to_pad > 0:
    last_row = df.iloc[-1]
    padding_data = np.tile(last_row.values, (num_to_pad, 1))
    padded_df = pd.DataFrame(padding_data, columns=df.columns)
    df = pd.concat([df, padded_df], axis=0)
    df = df.reset_index(drop=True)

small_dataset =df
probs_list=[]
for i in range(len(small_dataset)):
    
    if i>=len(small_dataset)-num_consecutive_frames:
        break
    
    
    oridata=small_dataset.loc[i:i+num_consecutive_frames-1,:].copy()
    oridata=oridata.reset_index(drop=True)
    data=[]
    # print(oridata)
    if str(oridata.loc[0,'pos'])=='nan':
            true_list.append(0)
    elif str(oridata.loc[0,'pos'])=='top':
            true_list.append(1)
    elif str(oridata.loc[0,'pos'])=='bottom':
            true_list.append(2)
        
    for index,row in oridata.iterrows():
        
        top=np.array(row['top']).reshape(-1,2)
        bottom=np.array(row['bottom']).reshape(-1,2)
        court=np.array(row['court']).reshape(-1,2)
        # net=np.array(row['net']).reshape(-1,2)
        ball=np.array(row['ball']).reshape(-1,2)
        
        frame_data = np.concatenate((top, bottom, court, ball), axis=0)
        
        if normalization:        
            # 将 x 坐标从 0 到 1920 映射到 1 到 2
            frame_data[:,0] /=1920

            # 将 y 坐标从 0 到 1080 映射到 1 到 2
            frame_data[:,1] /=1080

        data.append(frame_data.reshape(1,-1))
    data=np.array(data).reshape(1,-1)
    # print(data.shape)
    # if i%num_consecutive_frames!=0:
    #     final_list.append(0)
    #     continue     
    outputs=model(torch.FloatTensor(data).to(device))
    
    # print(outputs)
    pred=torch.argmax(outputs).item()
    probs=outputs[pred].item()
    final_list.append(pred)

print(true_list)
print(final_list)    


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
